In [1]:
####### dropout resnet18 vs without dropout
#### 
import torch
import sys
# sys.path.append("..")
sys.path.append("/home/yifan/projects/cophi/ContraVis")
import numpy as np

sys.path.append("/home/yifan/projects/deepdebugertool/DLVisDebugger")
# REF_PATH = "/home/yifan/Exp/Noise/0.2/experiment_vis"
# CONFIG_PATH = "/home/yifan/experiments/noise/20"
# TAR_PATH = "/home/yifan/dataset/cleanfornoise10"

# # CLEAN_PATH = "/home/yifan/Exp/Noise/0.2/experiment1"
TAR_PATH = "/home/yifan/dataset/resnet18_with_dropout/pairflip/cifar10/0"
REF_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"



ENCODER_DIMS=[512,256,256,256,256,2]
DECODER_DIMS= [2,256,256,256,256,512]
VIS_MODEL_NAME = 'vis2'

########## initulize reference data and target data
from singleVis.DataInit import DataInit
REF_EPOCH = 200
TAR_EPOCH = 200
DEVICE = "cuda:1"
tar_datainit = DataInit(TAR_PATH,TAR_PATH,TAR_EPOCH,DEVICE)
ref_datainit = DataInit(REF_PATH,REF_PATH,REF_EPOCH,DEVICE)

ref_model, ref_provider, ref_train_data, ref_prediction, ref_prediction_res, ref_scores = ref_datainit.getData()
tar_model, tar_provider, tar_train_data, tar_prediction, tar_prediction_res, tar_scores = tar_datainit.getData()




/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 431.96it/s]


NET resnet18_with_dropout
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 6561.84it/s]


In [2]:
from scipy.linalg import svd  
def orthogonal_mapping(data1, data2):
        """
        use Orthogonal Mapping, map data2 to data1's space。
    
        :param data1: numpy array, shape (n_samples, n_features)
        :param data2: numpy array, shape (n_samples, n_features)
        :return: data2_mapped: numpy array, mapped data2
        """
        # step1: Centralized data
        data1_centered = data1 - np.mean(data1, axis=0)
        data2_centered = data2 - np.mean(data2, axis=0)
        # step2: Calculate the cross-covariance matrix
        C = data2_centered.T @ data1_centered
        # step3: singular value decomposition
        U, _, Vt = svd(C)

        # step4: Compute orthogonal mapping matrix
        W = U @ Vt

        # step5: Apply mapping matrix
        data2_mapped = data2_centered @ W
    
        return data2_mapped
X_train1 = ref_provider.train_representation(REF_EPOCH)
X_train2 = tar_provider.train_representation(TAR_EPOCH)
X_train2 = X_train2.reshape(X_train2.shape[0],X_train2.shape[1])
print(X_train1.shape)
print(X_train2.shape)
data2_mapped = orthogonal_mapping(X_train1,X_train2)

(50000, 512)
(50000, 512)


In [7]:
from eval.evaluate import *
rate = evaluate_high_dimesion_trans_knn_preserving(data2_mapped, X_train2)
print(rate)

0.9999986666666667


In [4]:
torch.cuda.empty_cache()

In [9]:
from contrast.transfomration import TransformationTrainer

trainer = TransformationTrainer(X_train1,X_train2, DEVICE)
tarns_model,tar_mapped,ref_reconstructed  = trainer.transformation_train_advanced(num_epochs=30,lambda_similarity=0.1)
# tarns_model,tar_mapped,ref_reconstructed  = trainer.transformation_train()

Epoch [1/30], Loss: 1.4555
Epoch [21/30], Loss: 0.7849
Epoch [41/30], Loss: 0.6127
Epoch [61/30], Loss: 0.3983
Epoch [81/30], Loss: 0.2512
Epoch [101/30], Loss: 0.2071
Epoch [121/30], Loss: 0.1983
Epoch [141/30], Loss: 0.1954
Epoch [161/30], Loss: 0.1939
Epoch [181/30], Loss: 0.1923
Epoch [1/30],reconstruction_loss:0.0695,translation_loss:0.2431,neighbor_loss:7.5010, Loss: 7.6921
Epoch [2/30],reconstruction_loss:0.0697,translation_loss:0.2431,neighbor_loss:7.4974, Loss: 7.6886
Epoch [3/30],reconstruction_loss:0.0698,translation_loss:0.2430,neighbor_loss:7.4968, Loss: 7.6882
Epoch [4/30],reconstruction_loss:0.0698,translation_loss:0.2430,neighbor_loss:7.4928, Loss: 7.6841
Epoch [5/30],reconstruction_loss:0.0696,translation_loss:0.2429,neighbor_loss:7.4913, Loss: 7.6824
Epoch [6/30],reconstruction_loss:0.0694,translation_loss:0.2428,neighbor_loss:7.4881, Loss: 7.6789
Epoch [7/30],reconstruction_loss:0.0694,translation_loss:0.2427,neighbor_loss:7.4864, Loss: 7.6772
Epoch [8/30],reconstruc

In [10]:
ref_pred = ref_provider.get_pred(REF_EPOCH, X_train1).argmax(axis=1)
tar_pred = tar_provider.get_pred(TAR_EPOCH, X_train2).argmax(axis=1)

100%|██████████| 250/250 [00:00<00:00, 6616.33it/s]


In [13]:
from eval.evaluate import *
rate = evaluate_high_dimesion_trans_knn_preserving(tar_mapped, X_train2)
print(rate)

0.505016


In [11]:

reconstructed_pred = tar_provider.get_pred(TAR_EPOCH, ref_reconstructed).argmax(axis=1)

100%|██████████| 250/250 [00:00<00:00, 7398.41it/s]


In [12]:
diff = 0
for i in range(len(reconstructed_pred)):
    if reconstructed_pred[i] != tar_pred[i]:
        diff = diff+1
print(diff)

4


In [14]:
from sklearn.neighbors import NearestNeighbors

sub_len = 50000

# 1. use cos sim build 
model = NearestNeighbors(n_neighbors=1, metric='cosine', algorithm='auto').fit(tar_mapped[:sub_len])

# 2. find X_train1 most sim point data2_mapped中的最相似点
distances, most_similar_points = model.kneighbors(X_train1[:sub_len])

most_similar_points = most_similar_points.flatten()  # 将索引数组转为1D

# 3. 比较预测值
m = np.sum(ref_pred[:sub_len] != tar_pred[most_similar_points])
k = np.sum(ref_pred[:sub_len] != tar_pred[:sub_len])

print("diff pred number is {}".format(m))
print("index diff pred number is {}".format(k))


diff pred number is 41
index diff pred number is 12
